In [2]:
from prune_utils import *
from dataloaders import *
from scene_net import *
from evaluation import *
from quantus.metrics.faithfulness.irof import IROF
import quantus
import torch
from torch.utils.data import DataLoader
from dataloaders import *
from torch.autograd import Variable
import captum

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
print(os.getcwd())

d:\Documents\University\AI BSc\Year 3\Thesis\DiSparse-Multitask-Model-Compression


In [5]:
DATA_ROOT = 'data/nyu_v2/nyu_v2'
TASKS = ["seg", "sn"]
TASKS_NUM_CLASS = [40, 3]
IMAGE_SHAPE = (480, 640)

In [6]:
test_dataset = NYU_v2(DATA_ROOT, 'test')
test_loader = DataLoader(test_dataset, batch_size=1, num_workers=8, shuffle=True, pin_memory=True)

NYU_v2


In [8]:
model = SceneNet(TASKS_NUM_CLASS)

In [9]:
for module in model.modules():
    # Check if it's basic block
    if isinstance(module, nn.modules.conv.Conv2d) or isinstance(module, nn.modules.Linear):
        module = prune.identity(module, 'weight')

In [10]:
# 3. Load the state dictionary into your model
model.load_state_dict(torch.load('data/models/best_nyuv2_disparse_pt_90.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [11]:
irof_metric = quantus.IROF(segmentation_method='slic', abs=False, normalise=True)

In [12]:
irof_metric

In [13]:
#Prepare your data:
#Ensure you have your model
#input data (x_batch), 
#corresponding ground truth labels (y_batch), 
#and optionally pre-computed attributions (a_batch).

In [14]:
# # Iterate over batches in the dataloader
# for i, gt_batch in enumerate(test_loader):
#     print(i)
#     model.eval()
#     gt_batch["img"] = Variable(gt_batch["img"]).to(device)
#     if "seg" in gt_batch:
#         gt_batch["seg"] = Variable(gt_batch["seg"]).to(device)
#     if "depth" in gt_batch:
#         gt_batch["depth"] = Variable(gt_batch["depth"]).to(device)
#     if "normal" in gt_batch:
#         gt_batch["normal"] = Variable(gt_batch["normal"]).to(device)
#     if "keypoint" in gt_batch:
#         gt_batch["keypoint"] = Variable(
#             gt_batch["keypoint"]).to(device)
#     if "edge" in gt_batch:
#         gt_batch["edge"] = Variable(gt_batch["edge"]).to(device)

#     # Call the metric instance to produce scores.
#     results = {method: irof_metric(model=model, 
#                                     x_batch=gt_batch["img"],
#                                     y_batch=gt_batch["seg"],
#                                     a_batch=None,
#                                     device=device,
#                                     explain_func=captum.explain, 
#                                     explain_func_kwargs={"method": method}) for method in ["Saliency", "IntegratedGradients"]}

#     # Plot example!
#     #region_perturb.plot(results=results) 
#     results

In [15]:
print(torch.__version__)

2.1.1+cpu


In [20]:
# Iterate over batches in the dataloader
for i, gt_batch in enumerate(test_loader):
    print(i)
    model.eval()
    gt_batch["img"] = Variable(gt_batch["img"]).to(device)
    if "seg" in gt_batch:
        gt_batch["seg"] = Variable(gt_batch["seg"]).to(device)
    if "depth" in gt_batch:
        gt_batch["depth"] = Variable(gt_batch["depth"]).to(device)
    if "normal" in gt_batch:
        gt_batch["normal"] = Variable(gt_batch["normal"]).to(device)
    if "keypoint" in gt_batch:
        gt_batch["keypoint"] = Variable(
            gt_batch["keypoint"]).to(device)
    if "edge" in gt_batch:
        gt_batch["edge"] = Variable(gt_batch["edge"]).to(device)

    # Return IFOF scores in an one-liner - by calling the metric instance.
    scores = {method: quantus.IROF(segmentation_method="slic",
        perturb_baseline="mean",
        perturb_func=quantus.perturb_func.baseline_replacement_by_indices,
        return_aggregate=False,
    )(model=model,
    x_batch=gt_batch["img"],
    y_batch=gt_batch["seg"],
    a_batch=None,
    device=device,
    explain_func=quantus.explain, 
    explain_func_kwargs={"method": method}) for method in ["Saliency"]}

    scores # felzenswalb